In [1]:
import re
import time

import pandas as pd
import requests
from requests_html import HTML

In [2]:
base_url = "https://stackoverflow.com/questions/tagged/"
tag = "dask"
query_filter = "Newest"
url = f"{base_url}{tag}?tab={query_filter}"
url


'https://stackoverflow.com/questions/tagged/dask?tab=Newest'

In [7]:
r = requests.get(url)
html_str = r.text
html = HTML(html=html_str)

`.s-post-summary`: Base parent container for a post summary

https://stackoverflow.design/product/components/post-summary/

In [12]:
html.find(".s-post-summary")[:3]

[<Element 'div' id='question-summary-74688899' class=('s-post-summary', 'js-post-summary') data-post-id='74688899' data-post-type-id='1'>,
 <Element 'div' id='question-summary-74668680' class=('s-post-summary', 'js-post-summary') data-post-id='74668680' data-post-type-id='1'>,
 <Element 'div' id='question-summary-74662292' class=('s-post-summary', 'js-post-summary') data-post-id='74662292' data-post-type-id='1'>]

In [10]:
html.find(".s-post-summary")[0].text

'0 votes\n0 answers\n13 views\nIs py-datatable compatible with Dask?\nDoes Python Datatable work with distributed big data frameworks like Dask? I have plenty of data.table experience on R, but not on Python. For a person familiar with the tools this might sound a ...\ndask\npy-datatable\nJylpah\n183\nasked 3 hours ago'

In [13]:
print(html.find(".s-post-summary")[0].text)

0 votes
0 answers
13 views
Is py-datatable compatible with Dask?
Does Python Datatable work with distributed big data frameworks like Dask? I have plenty of data.table experience on R, but not on Python. For a person familiar with the tools this might sound a ...
dask
py-datatable
Jylpah
183
asked 3 hours ago


In [14]:
q1 = html.find(".s-post-summary")[0]
q1

<Element 'div' id='question-summary-74688899' class=('s-post-summary', 'js-post-summary') data-post-id='74688899' data-post-type-id='1'>

In [42]:
#get link
q1.find(".s-link", first=True)

<Element 'a' href='/questions/74688899/is-py-datatable-compatible-with-dask' class=('s-link',)>

In [26]:
# `.s-post-summary--stats`: Container for the post summary stats
q1.find(".s-post-summary--stats", first=True).text.replace("\n", " ")

'0 votes 0 answers 13 views'

In [27]:
# A container for post meta data, things like tags and user cards.
q1.find(".s-post-summary--meta", first=True).text.replace("\n", " ")


'dask py-datatable Jylpah 183 asked 3 hours ago'

In [28]:
# An optional content excerpt truncated at 2 lines.
q1.find(".s-post-summary--content-excerpt", first=True).text

'Does Python Datatable work with distributed big data frameworks like Dask? I have plenty of data.table experience on R, but not on Python. For a person familiar with the tools this might sound a ...'

In [34]:
#title
q1.find(".s-post-summary--content-title", first=True).text

'Is py-datatable compatible with Dask?'

Test richards code

In [43]:
questions = html.find(".s-post-summary")

In [50]:
key_class_dict = {"title": ".s-post-summary--content-title" ,
                  "stats": ".s-post-summary--stats",
                  "tags": ".s-post-summary--meta-tags"}

In [52]:
key_class_dict.items

<function dict.items>

In [54]:
datas = []

for q_el in questions:
    q_data = {}
    for k, v in key_class_dict.items():
        q_data[k] = q_el.find(v, first=True).text
    datas.append(q_data)

In [56]:
datas[:5]

[{'title': 'Is py-datatable compatible with Dask?',
  'stats': '0 votes\n0 answers\n13 views',
  'tags': 'dask\npy-datatable'},
 {'title': 'Dask (delayed) vs pandas/function returns',
  'stats': '0 votes\n0 answers\n14 views',
  'tags': 'pandas\ndask'},
 {'title': 'What is the most efficient way to access large dask arrays in functions submitted to the workers?',
  'stats': '0 votes\n0 answers\n12 views',
  'tags': 'python-3.x\ndask'},
 {'title': 'dask/core.py:119: RuntimeWarning: invalid value encountered in divide',
  'stats': '0 votes\n0 answers\n12 views',
  'tags': 'runtime\ndask\ndivide'},
 {'title': "Dask worker out of memory but I don't know why",
  'stats': '0 votes\n1 answer\n29 views',
  'tags': 'python-3.x\ndask'}]

In [57]:
# define keynames and quetions needed
# keynames = ["title", "stats", "tags"]
# classes_needed = [
#     ".s-post-summary--content-title",
#     ".s-post-summary--stats",
#     ".s-post-summary--meta-tags",
# ]


In [58]:
# datas = []

# for q_el in questions:
#     q_data = {}
#     for i, _class in enumerate(classes_needed):
#         sub_el = q_el.find(_class, first=True)
#         keyname = keynames[i]
#         q_data[keyname] = sub_el.text
#     datas.append(q_data)


In [60]:
len(datas)

50

In [61]:
df = pd.DataFrame(datas)
df.head(5)

,title,stats,tags
0,Is py-datatable compatible with Dask?,0 votes\n0 answers\n13 views,dask\npy-datatable
1,Dask (delayed) vs pandas/function returns,0 votes\n0 answers\n14 views,pandas\ndask
2,What is the most efficient way to access large...,0 votes\n0 answers\n12 views,python-3.x\ndask
3,dask/core.py:119: RuntimeWarning: invalid valu...,0 votes\n0 answers\n12 views,runtime\ndask\ndivide
4,Dask worker out of memory but I don't know why,0 votes\n1 answer\n29 views,python-3.x\ndask


In [76]:
# get only numbers
stats_test = re.findall(
    r"\d+", q1.find(".s-post-summary--stats", first=True).text
)
stats_test

['0', '0', '13']

In [81]:
q1.find(".s-post-summary--meta-tags", first=True).text.split("\n")

['dask', 'py-datatable']

In [101]:
re.findall(
    r"\d+", q1.find(".s-post-summary--stats", first=True).text
)

['0', '0', '13']

In [108]:
# function that will clean the scraped data
def clean_scraped_data(text, keyname=None):
    if keyname == "stats":
        text_strings = re.findall(r"\d+", text)
        
        try:
            text =[int(s) for s in text_strings]
        except ValueError:
            text =[int(re.sub("k", "000", s)) for s in text_strings]

    elif keyname == "tags":
        return text.split("\n")
    return text

In [109]:
datas = []

for q_el in questions:
    q_data = {}
    for k, v in key_class_dict.items():
        q_data[k] = clean_scraped_data(q_el.find(v, first=True).text, k)
    datas.append(q_data)

In [111]:
# df = pd.DataFrame(datas)
# df.head()


In [160]:
# define function that will parse a single page
def extract_url_and_parse_html(url):
    r = requests.get(url)
    if r.status_code not in range(200, 299):
        return []
    
    html = HTML(html=r.text)
        
    questions = html.find(".s-post-summary")
    
    key_class_dict = {"title": ".s-post-summary--content-title" ,
                  "stats": ".s-post-summary--stats",
                  "tags": ".s-post-summary--meta-tags"}

    datas = []

    for q_el in questions:
        q_data = {}
        for k, v in key_class_dict.items():
            q_data[k] = clean_scraped_data(q_el.find(v, first=True).text, k)
        
        q_data["vote"] = q_data["stats"][0]
        q_data["answer"] = q_data["stats"][1]
        q_data["view"] = q_data["stats"][2]
        datas.append(q_data)
    return datas

In [121]:
def scrape_tag(tag="python", query_filter="Newest", max_pages=10, pagesize=100):
    base_url = "https://stackoverflow.com/questions/tagged/"
    datas_page = []
    for p in range(max_pages):
        page_num = p + 1
        url = f"{base_url}{tag}?tab={query_filter}&page={page_num}&pagesize={pagesize}"
        datas_page += extract_url_and_parse_html(url)
        time.sleep(1.2) #need between requests.
    return datas_page

In [114]:
%%time
seq = scrape_tag(max_pages=10)


CPU times: user 2.53 s, sys: 99.2 ms, total: 2.63 s
Wall time: 16.3 s


In [116]:
len(seq)

500

In [122]:
t = scrape_tag(max_pages=1)

In [117]:
df = pd.DataFrame(seq)
df.head()

,title,stats,tags
0,Pandas Expanding Rows According to the Condition,"[0, 0, 6]","[python, pandas, dataframe]"
1,SSH to remote system and run python script and...,"[0, 0, 6]","[python, python-3.x, ssh, remote-access, scp]"
2,Docker-Django-Nginx problem with http/https,"[0, 0, 4]","[python, docker, http, nginx, https]"
3,Is list in python is mutable? [duplicate],"[4, 0, 17]",[python]
4,How to sort column wise based on custom values...,"[0, 0, 10]","[python, pandas, sorting, columnsorting]"


In [118]:
import dask.dataframe as dd

In [165]:
def scrape_single_tag(page):
    base_url = "https://stackoverflow.com/questions/tagged/"
    tag="dask"
    query_filter="Newest"
    pagesize=100
    datas_page = []

    url = f"{base_url}{tag}?tab={query_filter}&page={page}&pagesize={pagesize}"
    datas_page += extract_url_and_parse_html(url)
    #time.sleep(1.2) #need between requests.
    return pd.DataFrame(datas_page)

In [131]:
from dask.distributed import Client, wait

In [128]:
client = Client()

In [166]:
%%time
t = client.map(scrape_single_tag, range(10))
wait(t)

CPU times: user 103 ms, sys: 15.6 ms, total: 119 ms
Wall time: 949 ms


DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: scrape_single_tag-3976a22be77c2b9019662479a556ec31>, <Future: finished, type: pandas.core.frame.DataFrame, key: scrape_single_tag-5643478c4cc454ce1bb26c1794e3b7a6>, <Future: finished, type: pandas.core.frame.DataFrame, key: scrape_single_tag-ee10f11f1d0cc02e7a803faa2a949887>, <Future: finished, type: pandas.core.frame.DataFrame, key: scrape_single_tag-323dcf8e792172f9d0e7007df5cd00a2>, <Future: finished, type: pandas.core.frame.DataFrame, key: scrape_single_tag-0f4d46093a47c5144bf8a556170b9309>, <Future: finished, type: pandas.core.frame.DataFrame, key: scrape_single_tag-eaed47cf61308a314802180268cd2178>, <Future: finished, type: pandas.core.frame.DataFrame, key: scrape_single_tag-3afd382317d5656b3dad699c11016519>, <Future: finished, type: pandas.core.frame.DataFrame, key: scrape_single_tag-af48af8b5918ee794cc81fba5ff193d4>, <Future: finished, type: pandas.core.frame.DataFrame, key: scrape_single_tag

In [167]:
p1 = t[0].result()

In [168]:
p1.head()

,title,stats,tags,vote,answer,view
0,Is py-datatable compatible with Dask?,"[0, 0, 16]","[dask, py-datatable]",0,0,16
1,Dask (delayed) vs pandas/function returns,"[0, 0, 16]","[pandas, dask]",0,0,16
2,What is the most efficient way to access large...,"[0, 0, 12]","[python-3.x, dask]",0,0,12
3,dask/core.py:119: RuntimeWarning: invalid valu...,"[0, 0, 12]","[runtime, dask, divide]",0,0,12
4,Dask worker out of memory but I don't know why,"[0, 1, 29]","[python-3.x, dask]",0,1,29


In [172]:
t[0].result()[]

<Future: finished, type: pandas.core.frame.DataFrame, key: scrape_single_tag-0f4d46093a47c5144bf8a556170b9309>

In [188]:
del t15

In [169]:
from dask.distributed import as_completed

In [189]:
%%time
t15 = client.map(scrape_single_tag, range(15))

tot_views= 0 

for future in as_completed(t15):
    test = future.result()['view']
    tot_views += test.sum()

tot_views

CPU times: user 35.5 ms, sys: 7.28 ms, total: 42.8 ms
Wall time: 51.7 ms


105874

In [173]:
t

[<Future: finished, type: pandas.core.frame.DataFrame, key: scrape_single_tag-0f4d46093a47c5144bf8a556170b9309>,
 <Future: finished, type: pandas.core.frame.DataFrame, key: scrape_single_tag-eaed47cf61308a314802180268cd2178>,
 <Future: finished, type: pandas.core.frame.DataFrame, key: scrape_single_tag-af48af8b5918ee794cc81fba5ff193d4>,
 <Future: finished, type: pandas.core.frame.DataFrame, key: scrape_single_tag-3976a22be77c2b9019662479a556ec31>,
 <Future: finished, type: pandas.core.frame.DataFrame, key: scrape_single_tag-ee10f11f1d0cc02e7a803faa2a949887>,
 <Future: finished, type: pandas.core.frame.DataFrame, key: scrape_single_tag-323dcf8e792172f9d0e7007df5cd00a2>,
 <Future: finished, type: pandas.core.frame.DataFrame, key: scrape_single_tag-9546d0f6b603b7eed9792b393009782a>,
 <Future: finished, type: pandas.core.frame.DataFrame, key: scrape_single_tag-5643478c4cc454ce1bb26c1794e3b7a6>,
 <Future: finished, type: pandas.core.frame.DataFrame, key: scrape_single_tag-19e7e83096c0736547

In [183]:
test_del = dd.from_delayed(t15)

In [184]:
test_del

,title,stats,tags,vote,answer,view
npartitions=15,,,,,,
,object,object,object,int64,int64,int64
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [185]:
test_del['view'].sum()

dd.Scalar<series-..., dtype=int64>

In [186]:
%%time
test_del['view'].sum().compute()

CPU times: user 37.5 ms, sys: 4.54 ms, total: 42.1 ms
Wall time: 49 ms


105874